# Drum Transcriber (Demucs SOTA Edition) Colab

Run this notebook to launch the **SOTA Demucs-powered** Drum Transcriber Web UI.

**Features**:
- **Source Separation**: Uses Facebook's `Demucs` to isolate drums from the mix.
- **High Accuracy**: Transcription runs on the clean drum track, significantly improving hit detection.

In [ ]:
# @title 1. Setup Environment
# @markdown Installs dependencies and Demucs.

import os

# Install Demucs and dependencies
!pip install demucs gradio yt-dlp librosa pandas plotly scikit-learn soundfile

# Clone Repo
repo_url = "https://github.com/AgentHitmanFaris/DrumTranscriber.git"
repo_name = "DrumTranscriber"

if os.path.exists(repo_name):
    %cd {repo_name}
    !git pull
else:
    !git clone {repo_url}
    %cd {repo_name}

# Download our lightweight transcription model
!python -c "from utils.config import download_model; download_model()"


In [ ]:
# @title 2. Run App (Demucs Mode)
# @markdown This launches the app, injecting the Demucs separation step before transcription.

launcher_code = """
import gradio_app
import librosa
from demucs_processing import DemucsSeparator

# Initialize Separator
separator = DemucsSeparator()

# Save original process_audio to keep the old signature/logic if needed
original_process_audio = gradio_app.process_audio

# Monkey-patch process_audio to intercept the file
def patched_process_audio(audio_file, start_time, duration=30, progress=gradio_app.gr.Progress()):
    if not audio_file:
        return None, None, None, "No audio file provided."
    
    progress(0.1, desc="Separating Drums with Demucs (SOTA)...")
    
    # Run Separation
    try:
        drums_path = separator.separate(audio_file)
        if drums_path:
            # If separation succeeds, transcribe the DRUMS track
            print(f"Transcribing separated drums: {drums_path}")
            target_file = drums_path
        else:
            # Fallback to original if separation fails
            print("Separation failed, using original mix.")
            progress(0.2, desc="Separation failed, using mix...")
            target_file = audio_file
    except Exception as e:
        print(f"Demucs error: {e}")
        target_file = audio_file

    # Call original processing logic with the (possibly separated) file
    # Note: original_process_audio loads the file with librosa.
    # We just pass the new path.
    return original_process_audio(target_file, start_time, duration, progress)

# Apply Patch
gradio_app.process_audio = patched_process_audio

# Launch
gradio_app.demo.launch(share=True)
"""

with open('run_demucs.py', 'w') as f:
    f.write(launcher_code)

!python run_demucs.py
